In [16]:
from PyFoam.Execution.AnalyzedRunner import AnalyzedRunner
from PyFoam.RunDictionary.SolutionDirectory import SolutionDirectory
from PyFoam.LogAnalysis.StandardLogAnalyzer import StandardLogAnalyzer

import os
from pathlib import Path
import numpy as np

path = Path("./elbow").absolute()
case = "test"
solver = "icoFoam"

dire = SolutionDirectory(path)

In [18]:
from PyFoam.RunDictionary.ParameterFile import ParameterFile

controlDict = ParameterFile(dire.controlDict())

In [21]:
controlDict.readParameter('endTime')

'10'

In [27]:
from PyFoam.RunDictionary.ParsedParameterFile import ParsedParameterFile

In [28]:
controlDict = ParsedParameterFile(dire.controlDict())

In [30]:
controlDict.content

{'application': 'icoFoam',
 'startFrom': 'latestTime',
 'startTime': 0,
 'stopAt': 'endTime',
 'endTime': 10,
 'deltaT': 0.05,
 'writeControl': 'timeStep',
 'writeInterval': 20,
 'purgeWrite': 0,
 'writeFormat': 'ascii',
 'writePrecision': 6,
 'writeCompression': off,
 'timeFormat': 'general',
 'timePrecision': 6,
 'runTimeModifiable': true}

In [39]:
controlDict['endTime'] = 30

In [41]:
controlDict.writeFile()

In [42]:
!cat ./elbow/system/controlDict

// -*- C++ -*-
// File generated by PyFoam - sorry for the ugliness

FoamFile
{
 version 2.0;
 format ascii;
 class dictionary;
 location "system";
 object controlDict;
}

application icoFoam;

startFrom latestTime;

startTime 0;

stopAt endTime;

endTime 30;

deltaT 0.05;

writeControl timeStep;

writeInterval 20;

purgeWrite 0;

writeFormat ascii;

writePrecision 6;

writeCompression no;
timeFormat general;

timePrecision 6;

runTimeModifiable yes;


In [6]:
os.chdir(orig_path)

In [ ]:
from PyFoam.Execution.ConvergenceRunner import ConvergenceRunner
from PyFoam.Execution.UtilityRunner import UtilityRunner
from PyFoam.LogAnalysis.BoundingLogAnalyzer import BoundingLogAnalyzer
from PyFoam.RunDictionary.SolutionFile import SolutionFile
from PyFoam.RunDictionary.SolutionDirectory import SolutionDirectory
 
solver="icoFoam"
case="c1"
pCmd="calcPressureDifference"
mCmd="calcMassFlow"
 
dire=SolutionDirectory(case,archive="InletVariation")
dire.clearResults()
dire.addBackup("PyFoamSolve.logfile")
dire.addBackup("PyFoamSolve.analyzed")
dire.addBackup("Pressure.analyzed")
dire.addBackup("MassFlow.analyzed")
 
sol=SolutionFile(dire.initialDir(),"U")
 
maximum=1.
nr=10
 
f=dire.makeFile("InflowVariationResults")
 
for i in range(nr+1):
    # Set the boundary condition at the inlet
    val=(maximum*i)/nr
    print "Inlet velocity:",val
    sol.replaceBoundary("inlet","(%f 0 0)" %(val))
 
    # Run the solver
    run=ConvergenceRunner(BoundingLogAnalyzer(),argv=[solver,".",case],silent=True)
    run.start()
    
    print "Last Time = ",dire.getLast()
 
    # Get the pressure difference (Using an external utility)
    pUtil=UtilityRunner(argv=[pCmd,".",case],silent=True,logname="Pressure")
    pUtil.add("deltaP","Pressure at .* Difference .*\] (.+)")
    pUtil.start()
 
    deltaP=pUtil.get("deltaP")[0]
 
    # Get the mass flow
    mUtil=UtilityRunner(argv=[mCmd,".",case,"-latestTime"],silent=True,logname="MassFlow")
    mUtil.add("mass","Flux at (.+?) .*\] (.+)",idNr=1)
    mUtil.start()
 
    massFlow=mUtil.get("mass",ID="outlet")[0]
 
    # Archive the results
    dire.lastToArchive("vel=%g" % (val))
 
    # Clear results
    dire.clearResults()
 
    # Output current stuff
    print "Vel: ",val,"DeltaP: ",deltaP,"Mass Flow:",massFlow
    f.writeLine( (val,deltaP,massFlow) )
    
sol.purgeFile()